In [1]:
import pandas as pd
import numpy as np

In [2]:
import re
import unidecode
import numpy as np

def process_string_1(s):
    # Remove espaços em branco extras
    s = str(s).strip()
    
    numbers = re.findall(r'\d+(?:\.\d+)?', s)
    if numbers == []:
        numbers.append(np.nan)
    
    has_parentheses = 1 if '(' in s or ')' in s else 0
    
    letters = ''.join(re.findall(r'[a-zA-Z]', s))
    if letters == '':
        letters = 0
    return numbers, has_parentheses, letters

def process_string(s):
    # Remove espaços em branco extras
    s = str(s).strip()
    
    numbers = re.findall(r'\d+(?:\.\d+)?', s)
    if numbers == []:
        numbers.append(np.nan)
    
    has_parentheses = 1 if '(' in s or ')' in s or '-' in s else 0
    
    letters = ''.join(re.findall(r'[a-zA-Z]', s))
    if letters == '':
        letters = 0
    return numbers, has_parentheses, letters

In [3]:
column_order = ['ID Resultado', 'ID Competidor', 'Nome Competidor',
                'ID Competição', 'Classe Vela', 'Pontuação Regata',
                'Descarte', 'Flotilha', 'Posição Geral', 'Punição',
                'Pontuação Total', 'Nett', 'Nome Competição', 'Regata']

In [4]:
df1 = pd.read_csv("../scraped-data-2024/cluster_1_resolvido.csv")
df1.drop(["Unnamed: 0", "Nacionalidade"], axis=1, inplace=True)
df1.rename(columns={'Athlete': 'Nome Competidor', "Net": "Nett", "Total": 'Pontuação Total', "Position":'Posição Geral'}, inplace=True)
df1[['ID Resultado', 'ID Competidor', 'ID Competição', 'Descarte', 'Punição', 'Flotilha']] = ''

In [5]:
df1['Punição'] = df1['Pontuação Regata'].apply(lambda x: process_string_1(x)[2])
df1['Descarte'] = df1['Pontuação Regata'].apply(lambda x: process_string_1(x)[1])
df1['Pontuação Regata'] = df1['Pontuação Regata'].apply(lambda x: float(process_string_1(x)[0][0]))

In [6]:
for i, data1 in df1.iterrows():
    # if data1["Classe Vela"] in ['IQFOIL FEM.', 'IQFOIL MASC.', 'FORMULA KITE FEM.',
    #    'FORMULA KITE MASC.', '49ER', '49ERFX', 'NACRA 17']:
    if data1["Classe Vela"] in ['IQFOIL FEM.', 'IQFOIL MASC.']:
        nome = data1['Nome Competidor'].split('/')[0].strip()
        nomes = nome.split(',')
        match len(nomes):
            case 1:
                nome = nomes[0]
            case 2:
                nome = nomes[1].strip() + ' ' + nomes[0].strip()
            case 3:
                nome = nomes[-1].strip() + " " + nomes[0].strip() + ", " + nomes[1].strip().replace('\r\n', ' ').replace('?', '')
        
    elif data1["Classe Vela"] == 'FORMULA KITE FEM.':
        nome = data1['Nome Competidor'].replace('1/2O', '').replace('1/2', '').replace('?', '').replace("  ", " ")
        nomes = nome.strip().split(' ')
        match len(nomes):
            case 1:
                nome = nomes[0].strip()
            case 2:
                nome = nomes[1].strip() + " " + nomes[0].strip()
            case 3:
                nome = nomes[2].strip() + " " + nomes[0].strip() + " " + nomes[1].strip()
            case _:
                nome = nomes[-1].strip() + " " + nomes[0].strip() + " " + nomes[1].strip()

    elif data1["Classe Vela"] == 'FORMULA KITE MASC.':
        nome = data1['Nome Competidor'].replace('1/2S', '').replace('1/2', '').replace('?', '').replace("  ", " ").replace("  ", " ")
        nomes = nome.strip().split(' ')
        match len(nomes):
            case 2:
                nome = nomes[1].strip() + " " + nomes[0].strip()
            case 3:
                nome = nomes[-1].strip() + " " + nomes[0].strip() + " " + nomes[1].strip()
            case 4:
                nome = nomes[1].strip() + " " + nomes[2].strip() + " " + nomes[3].strip() + " " + nomes[0].strip()

    elif data1["Classe Vela"] == '49ERFX':
        nome = data1['Nome Competidor'].strip().replace(', ,', ",").replace(",,", ",").replace('TPK\r\n', '').replace('SR\r\n', '').replace('\r\n', '')
        nomes = nome.split("/")
        if len(nomes) > 2:
            if nomes[2] == '':
                nomes = nomes[:1]
            else:
                nomes = [nomes[0], nomes[-1]]
        if len(nomes) == 1:
            nomes = nomes[0].strip().split(',')
        
        match len(nomes):
            case 2:
                name1, name2 = nomes
                name1 = name1.split(',')
                name2 = name2.split(',')
                nome = name1[-1].strip() + ' ' + name1[0].strip() + ', ' + name2[-1].strip() + ' ' + name2[0].strip()

    elif data1["Classe Vela"] == '49ER':
        nome = data1['Nome Competidor']

    else:
        nomes = data1['Nome Competidor'].strip().replace('?', '').replace('\r\n', ' / ').split('/')
        match len(nomes):
            case 1:
                nomes = nomes[0].split(',')
                if len(nomes) == 2:
                    nome = nomes[-1].strip() + ' ' + nomes[0].strip()
                else:
                    nome = nomes[-1].strip() + ' ' + nomes[0].strip() + ', ' + nomes[1].strip()
            case 2:
                nome = nomes[0].split(',')[1].strip() + ' ' + nomes[0].split(',')[0].strip() + ', ' + nomes[1].split(',')[1].strip() + ' ' + nomes[1].split(',')[0].strip()

            case _:
                if len(nomes) == 3:
                    nomes = [nomes[0], nomes[-1]]
                else:
                    nomes = [nomes[0], nomes[2]]
                nome = nomes[0].split(',')[1].strip() + ' ' + nomes[0].split(',')[0].strip() + ', ' + nomes[1].split(',')[1].strip() + ' ' + nomes[1].split(',')[0].strip()
    
    df1.loc[i, 'Nome Competidor'] = nome

In [7]:
df1["Classe Vela"].unique()

array(['IQFOIL FEM.', 'IQFOIL MASC.', 'FORMULA KITE FEM.',
       'FORMULA KITE MASC.', '49ER', '49ERFX', 'NACRA 17'], dtype=object)

In [8]:
df1[df1['Classe Vela'] == 'IQFOIL FEM.']['Nome Competidor'].unique()

array(['MINA MOBEKK', 'SHARON KANTOR', 'EMMA WILSON', 'HELENE NOESMOEN',
       'SHAHAR TIBI', 'PILAR LAMADRID TRUEBA', 'PALMA ?ARGO',
       'SARA WENNEKES', 'LILIAN DE GEUS', 'KATY SPYCHAKOV',
       'DELPHINE COUSIN', 'TAMAR STEINBERG', 'LOLA SORIN', 'ISLAY WATSON',
       'MAJA DZIARNOWSKA', 'MARTA MAGGETTI', 'HELLE OPPEDAL',
       'MAYA MORRIS', 'NICOLE VAN DER VELDEN', 'SASKIA SILLS',
       'MARIA BELEN BAZO GERMAN', 'THERESA MARIE STEINLEIN',
       'DANIELA PELEG', 'LENA ERDIL', 'VEERLE TEN HAVE', 'XIALING TAN',
       'COUTURIER MARION', 'SHAHAR RESHEF', 'XIANTING HUANG',
       ' AGUILAR CHAVEZ PEON', 'ALICE READ', 'MANJIA ZHENG',
       'MAJA KUCHTA', 'MANON PIANAZZA', 'KRISTYNA PINOSOVA',
       'GIORGIA SPECIALE', 'KWAN CHING MA',
       'ZOFIA TERESA NOCETI KLEPACKA', 'ADI MILLO', 'NAAMA GAZIT',
       'INGRID PUUSTA', 'ALISA ENGELMANN', 'CATRIN WILLIAMS',
       'DOMINIQUE STATER', 'LINA EREN', 'LUCIE BELBEOCH',
       'LAERKE BUHL-HANSEN', 'TUVA OPPEDAL', 'RENNA SOFIA

In [9]:
df1 = df1[column_order]
df1.to_csv('../cleaned_data_2024/cluster1.csv', index=False)

In [ ]:
df2 = pd.read_csv('../scraped-data-2024/cluster2-html.csv')
df2 = pd.concat([df2, pd.read_csv('../scraped-data-2024/cluster2_5-pdf.csv')])

In [ ]:
df2.replace('\xa0', np.nan, inplace=True)
df2.dropna(subset=["Pontuação Regata"], inplace=True)
df2['Punição'] = 0
df2['Descarte'] = 0

In [ ]:
df2['Punição'] = df2['Pontuação Regata'].apply(lambda x: process_string(x)[2])
df2['Descarte'] = df2['Pontuação Regata'].apply(lambda x: process_string(x)[1])
df2['Pontuação Regata'] = df2['Pontuação Regata'].apply(lambda x: process_string(x)[0][0])
df2["Nome Competidor"] = df2["Nome Competidor"] .apply(lambda x: unidecode.unidecode(x).upper())

In [ ]:
df2['ID Resultado'] = ''
df2['ID Competidor'] = ''

In [ ]:
df2 = df2[column_order]

In [ ]:
df2.to_csv('../cleaned_data_2024/cluster_2.csv', index=False)

In [ ]:
df3 = pd.read_csv('../scraped-data-2024/cluster3.csv')

In [ ]:
df3["Nome Competidor"] = df3["Nome Competidor"].apply(lambda x: str(x).strip().replace('/', ', '))
df3["Nome Competidor"] = df3["Nome Competidor"].apply(lambda x: unidecode.unidecode(x).upper())

In [ ]:
ids = ['ID Resultado', 'ID Competidor', 'ID Competição']
for id in ids:
    df3[id] = ''

In [ ]:
df3 = df3[column_order]

In [ ]:
df3.to_csv('../cleaned_data_2024/cluster_3.csv', index=False)

In [ ]:
df6 = pd.read_csv('../scraped-data-2024/cluster6-pdf.csv')

In [ ]:
df6["Nome Competidor"] = df6["Nome Competidor"].apply(lambda x: unidecode.unidecode(x).upper())
df6['Descarte'] = 0
df6[['ID Resultado', 'ID Competidor']] = ''

In [ ]:
df6 = df6[column_order]

In [ ]:
df6.to_csv('../cleaned_data_2024/cluster6.csv', index=False)

In [ ]:
df7 = pd.read_csv('../scraped-data-2024/cluster7-pdf.csv')

In [ ]:
def clean_names_7(line):
    results = line.strip().split(' - ')
    if len(results) > 1:
        punicao = results[0].strip("[")

        if punicao == results[0]:
            descarte = 0
            val = results[1].strip()

        else:
            descarte = 1
            val = results[1].strip("]")
    else:
        punicao = 0
        val = line.strip('[').strip(']')
        if val == line:
            descarte = 0
        else:
            descarte = 1

    return descarte, punicao, val

In [ ]:
df7['Descarte'] = df7['Pontuação Regata'].apply(lambda x: clean_names_7(x)[0])
df7['Punição'] = df7['Pontuação Regata'].apply(lambda x: clean_names_7(x)[1])
df7['Pontuação Regata'] = df7['Pontuação Regata'].apply(lambda x: clean_names_7(x)[2])

In [ ]:
df7['Nome Competidor'] = df7['Nome Competidor'].apply(lambda x: unidecode.unidecode(x).upper())
df7[['ID Resultado', 'ID Competidor']] =  ''

In [ ]:
df7 = df7[column_order]

In [ ]:
df7.to_csv('../cleaned_data_2024/cluster7.csv', index=False)

In [ ]:
df8 = pd.read_csv('../scraped-data-2024/cluster8-html.csv')

In [ ]:
df8 = df8.dropna(subset=['Pontuação Regata'])
df8["Nome Competidor"] = df8["Nome Competidor"].apply(lambda x: unidecode.unidecode(str(x)))

In [ ]:
def sep_nomes(nomes):
    nomes_list = str(nomes).strip().split(" ")
    new_nomes_list = list()

    for i in range(len(nomes_list)):
        if nomes_list[i] != nomes_list[i].title() and nomes_list[i] != nomes_list[i].upper():
            nnomee = ''
            nome = nomes_list[i]
            for i in range(len(nome)-1):
                if nome[i] == nome[i].upper() and nome[i+1] == nome[i+1].lower():
                    nnomee += ', '

                nnomee += nome[i]

            nnomee += nome[-1]
            nnomee += ' '

            new_nomes_list.append(nnomee)
        else:
            new_nomes_list.append(nomes_list[i])
            new_nomes_list.append(' ')

    new_nome = ''
    for name in new_nomes_list:
        new_nome += name
    
    return new_nome

In [ ]:
df8['Nome Competidor'] = df8['Nome Competidor'].apply(lambda x: sep_nomes(x).strip())
df8['Nome Competidor'] = df8['Nome Competidor'].apply(lambda x: unidecode.unidecode(x).upper())

In [ ]:
def get_descarte(line):
    descarte = 0
    if type(line) == str:
        if str(line).startswith('(') or str(line).endswith(')'):
            descarte = 1
    val = str(line).strip('(').strip(')')

    return descarte, val

In [ ]:
df8['Descarte'] = df8['Pontuação Regata'].apply(lambda x: get_descarte(x)[0])
df8['Pontuação Regata'] = df8['Pontuação Regata'].apply(lambda x: get_descarte(x)[1])
df8[['ID Resultado', 'ID Competidor']] = ''

In [ ]:
df8 = df8[column_order]

In [ ]:
df8.to_csv('../cleaned_data_2024/cluster8.csv', index=False)

In [ ]:
df9 = pd.read_csv('../scraped-data-2024/cluster9-html.csv')

In [ ]:
df9 = df9.dropna(subset=['Pontuação Regata'])
df9["Nome Competidor"] = df9["Nome Competidor"].apply(lambda x: unidecode.unidecode(str(x)))
df9['Nome Competidor'] = df9['Nome Competidor'].apply(lambda x: sep_nomes(x).strip())
df9['Nome Competidor'] = df9['Nome Competidor'].apply(lambda x: unidecode.unidecode(x).upper())
df9[['ID Resultado', 'ID Competidor']] = ''

In [ ]:
df9 = df9[column_order]

In [ ]:
df9.to_csv('../cleaned_data_2024/cluster9.csv', index=False)

In [ ]:
df14 = pd.read_csv('../scraped-data-2024/cluster14-html.csv')

In [ ]:
df14 = df14.dropna(subset=['Pontuação Regata'])
df14["Nome Competidor"] = df14["Nome Competidor"].apply(lambda x: unidecode.unidecode(str(x)))

In [ ]:
df14.to_csv('../cleaned_data_2024/cluster14.csv', index=False)

ID Resultado, ID Competidor, Nome Competidor, ID Competição, Classe Vela, Pontuação Regata, Descarte, Flotilha, Posição Geral, Punição, Pontuação Total, Nett, Nome Competição, Regata


In [ ]:
import os
import pandas as pd
files = [f for f in os.listdir('../cleaned_data_2024') if f.endswith('.csv')]

df_init = pd.read_csv("../cleaned_data_2024/"+files[0])
for file in files[1:]:
    df_init = pd.concat([df_init, pd.read_csv("../cleaned_data_2024/"+file)])
df_init.to_csv("../cleaned_data_2024/final_data.csv", index = False)    